# Nettoyer les données d'OpenFoodFacts

Nous allons explorer et nettoyer le jeu de données d'OpenFoodFacts, filtré dans le notebook précédent pour ne conserver que
* les fiches de produits vendus en France,
* les champs remplis pour au moins 1000 produits,
* les fiches contenant un nom de produit et une marque,
* les fiches marquées "complètes".

L'application successive de ces filtres nous permet d'explorer un sous-ensemble du jeu de données de départ:
90000 fiches avec 101 champs sélectionnées parmi 1.1 millions de fiches avec 178 champs.

La méthode la plus simple pour explorer le contenu du jeu de données est d'utiliser un outil graphique comme OpenRefine, qui permet aussi et surtout de nettoyer un jeu de données de façon reproductible.

OpenRefine étant plus adapté au travail sur des jeux de données un peu plus petits, nous commençons par sélectionner et exporter un sous-ensemble du jeu de données filtré.

## Sélection d'un sous-ensemble

Si nous voulions améliorer méthodiquement la qualité des données d'OpenFoodFacts, le plus efficace serait de travailler indépendamment sur des sous-ensembles cohérents de lignes et de colonnes, puis de fusionner tous les sous-ensembles nettoyés pour produire une version propre du jeu de données.
Pour cela, il faudrait créer plusieurs fichiers CSV contenant chacun un identifiant unique, ici le code-barres ou l'URL, ainsi qu'un petit nombre de colonnes liées entre elles, comme le nom, la marque et la quantité du produit, ou le nom, la marque et les valeurs nutritionnelles courantes.

Par manque de temps, nous n'allons pas être aussi méthodiques et nous allons simplement sélectionner 1000 fiches de produits laitiers.

In [1]:
import pandas as pd

In [2]:
# modifier le chemin vers le fichier CSV si nécessaire
# ici on utilise le fichier filtré dans le notebook précédent
CSV_FILE = '../data/off_subset.csv'

In [3]:
# on charge les types de données qu'on a spécifiés dans le notebook 1
DTYPE_FILE = '../data/dtype.txt'
with open(DTYPE_FILE) as f:
    dtype = eval(f.read())

In [4]:
df_off = pd.read_csv(CSV_FILE, sep='\t', dtype=dtype)
df_off.info(memory_usage='deep')

/home/mathieu/miniconda3/envs/off-explore/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90738 entries, 0 to 90737
Columns: 101 entries, code to nutrition-score-uk_100g
dtypes: category(18), float16(44), int64(2), object(37)
memory usage: 337.0 MB


In [5]:
# on prend les 1000 premières fiches
df_sub = df_off[df_off['categories_tags'].str.contains('en:dairies')][:1000]
print(df_sub.shape)
# on fait un dump CSV du jeu de données filtré
SEL_FILE = '../data/off_small.csv'
df_sub.to_csv(SEL_FILE, sep='\t', index=False)

(1000, 101)


## Présentation et installation d'OpenRefine

OpenRefine est un outil graphique, open source, pour travailler sur des jeux de données sales.
Cet outil permet de nettoyer un jeu de données de façon semi-automatique, par itérations successives.
OpenRefine stocke l'historique des opérations de nettoyage que vous appliquez et vous permet de l'exporter pour le rejouer, sur le même jeu de données ou sur une autre version de ce jeu de données.

Je vous conseille de visionner les 3 captures vidéo de la documentation officielle, qui donne une bonne idée de ce que vous pouvez réaliser avec OpenRefine.
https://github.com/OpenRefine/OpenRefine/wiki/Screencasts

Vous pouvez télécharger OpenRefine depuis cette page :
https://openrefine.org/download.html

## Lancement d'OpenRefine

Une fois que vous avez téléchargé et installé OpenRefine, vous pouvez ouvrir un fichier CSV, ici l'export du sous-ensemble de données réalisé précédemment.

Par défaut, OpenRefine s'alloue 1 Go de RAM, ce qui peut se réléver insuffisant.
Il faut alors modifier les paramètres de lancement d'OpenRefine pour lui allouer plus de RAM : https://github.com/OpenRefine/OpenRefine/wiki/FAQ:-Allocate-More-Memory

Par exemple sous linux :
```sh
# 4096 Mo = 4 Go de RAM
./refine -m 4096m
```

## Utilisation d'OpenRefine

Ouvrir le fichier.

Dans la fenêtre de configuration des options de parsing:
- vérifier que le séparateur (tab), la ligne d'en-tête... sont correctement détectés ;
- cocher la case "Parse cell text into numbers, dates..." pour mieux visualiser et manipuler ces types de données ;
- appuyer sur "create project" (en haut à droite).

L'import des données ne prend normalement que quelques secondes.

Si le temps d'import estimé augmente et se compte en minutes, c'est que vous n'avez pas alloué suffisamment de RAM ou que vous essayez de charger un fichier trop gros (retour aux étapes précédentes).

Nous pouvons créer une facette pour examiner une colonne, par exemple la colonne contenant les marques.

La facette met en évidence que le jeu de données contient de nombreuses variantes de graphie pour certaines marques.
L'utilisation d'une méthode de clustering dans une facette permet de regrouper et fusionner ces variantes par paquets.

Par exemple, combien y a-t-il de variantes pour la marque "Nos régions ont du talent, Leclerc" ?

Quelles sont les informations disponibles pour chaque cluster ?

Pour la méthode de clustering "key collision", quelles "keying functions" sont disponibles?

Comment fonctionnent-elles?

Produisent-elles des clusterings similaires?

Pourquoi?

L'historique des opérations de nettoyage, comme l'harmonisation de valeurs par clustering, peut être exporté afin d'être appliqué à un autre sous-ensemble du même jeu de données, ou à une version ultérieure du même jeu de données.

Copiez-collez ici l'historique des opérations que vous avez appliquées.